In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer
from gliner import GLiNER
import joblib

## Modelo Analisis de sentimientos

In [ ]:
dataset_sentimientos = pd.read_csv('./databases/sentimientos.csv')

In [ ]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1') 

labels = [(0, "Melancolico"), (1, "Ni fu Ni fa"), (2, "Alegre")]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(dataset_sentimientos['frase'].to_list(), dataset_sentimientos['clase'].to_list(), test_size=0.2, shuffle=True,random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train)
X_test_vectorized = model.encode(X_test)
# Creación y entrenamiento del modelo de Regresión Logística
modelo_sentimientos = LogisticRegression(max_iter=100, solver='lbfgs')
modelo_sentimientos.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_sentimientos = modelo_sentimientos.predict(X_test_vectorized)
acc_sentimientos = accuracy_score(y_test, y_pred_sentimientos)
report_sentimientos = classification_report(y_test, y_pred_sentimientos, zero_division=1)

print("Precisión Regresión Logística:", acc_sentimientos)
print("Reporte de clasificación Regresión Logística:\n", report_sentimientos)


## Modelo Recomendaciones

In [ ]:
modelo_recomendaciones = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
modelo_gliner = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

## Importamos los modelos

In [ ]:
joblib.dump(modelo_sentimientos, './models/modelo_sentimientos.pkl')
joblib.dump(model, './models/modelo_embedding.pkl')
joblib.dump(modelo_recomendaciones, './models/modelo_recomendaciones.pkl')
joblib.dump(modelo_gliner, './models/modelo_gliner.pkl')
print("Modelos exportados correctamente")